<a href="https://colab.research.google.com/github/kikiru328/CAKD3-Github/blob/main/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/transformers/

- HuggingFace는 자연어 처리 인공지능 모델에서, BERT 모델 같은 트랜스포머 모델들을 쉽게 다룰 수 있게 해주는 패키지입니다.
- 기본적으로 pytorch 기반으로 만들어져 있지만, 텐서플로우 2.0에서도 본 패키지 사용 가능합니다.
- 텐서플로우 2.0은 기존 케라스를 포함하고 있기 때문에, 기존 텐서플로우나 케라스에 익숙하신 분들이 쉽게 사용할 수 있습니다.
- 텐서플로우 2.0 기반의 huggingface 사용 방법을 네이버 영화 긍부정 분석을 실슴하면서 배워 보도록 하겠습니다.

* 인스톨
huggingface 패키지를 Colab에 설치합니다.

- 허깅페이스는 트랜스포머를 기반으로 하는 다양한 모델 (transformer.models)과 학습 스크립트(transformer.Trainer)를 구현해 놓은 모듈이다. 
- 원래는 파이토치로 layer, model등을 선언해주고 학습 스크립트도 전부 구현해야 하지만, 허깅 페이스를 사용하면 이런 수고를 덜 수 있다.
- 정리하면 허깅페이스라는 회사가 만든 transformers 패키지가 있고, 일반적인 파이토치 구현체의 layer.py, model.py이 transfomer.models 에 train.py가 transformer.Trainer에 대응된다

In [ ]:
# !pip install transformers
# !pip install sentencepiece

In [41]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
from tqdm import tqdm
import os

In [6]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [7]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 23.89 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [8]:
os.listdir('nsmc')

['code',
 'README.md',
 'raw',
 '.git',
 'ratings.txt',
 'ratings_train.txt',
 'synopses.json',
 'ratings_test.txt']

In [11]:
train = pd.read_table('nsmc/' + 'ratings_train.txt')
test = pd.read_table('nsmc/' + 'ratings_test.txt')
train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [18]:
# bert input
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [19]:
tokenizer.encode('보는내내 그대로 들어맞는 예측 카리스마 없는 악역')

[101,
 9356,
 11018,
 31605,
 31605,
 110589,
 71568,
 118913,
 11018,
 9576,
 119281,
 9786,
 79940,
 23811,
 40364,
 9520,
 23160,
 102]

In [20]:
tokenizer.tokenize('보는내내 그대로 들어맞는 예측 카리스마 없는 악역')

['보',
 '##는',
 '##내',
 '##내',
 '그대로',
 '들어',
 '##맞',
 '##는',
 '예',
 '##측',
 '카',
 '##리스',
 '##마',
 '없는',
 '악',
 '##역']

In [25]:
print( tokenizer.encode('전율을 일으키는 영화. 다시 보고 싶은 영화', max_length = 129 ,pad_to_max_length = True))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[101, 9665, 119183, 10622, 9641, 119185, 66815, 42428, 119, 25805, 98199, 9495, 10892, 42428, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [30]:
# mask 인풋
valid_num = len(tokenizer.encode('전율을 일으키는 영화. 다시 보고 싶은 영화'))
print(valid_num * [1] + (128 -valid_num) * [0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# 네이버 영화 평가 무장들을 버트 인풋으로 변환

In [38]:
def convert_data(data_df):
  global tokenizer

  SEQ_LEN= 128  # bert 인풋의 길이 

  tokens, masks, segments, targets = [],[],[],[]

  for i in tqdm(range(len(data_df))):
    token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length= SEQ_LEN, truncation=True,padding='max_length')

    num_zeros= token.count(0)
    mask = [1]* (SEQ_LEN-num_zeros) + [0]*num_zeros

    segment = [0] *SEQ_LEN

    tokens.append(token)
    masks.append(mask)
    segments.append(segment)

    targets.append(data_df[LABEL_COLUMN][i])

  tokens= np.array(tokens)
  masks= np.array(masks)
  segments= np.array(segments)
  targets= np.array(targets)

  return  [tokens,masks, segments], targets


In [50]:
def load_data(pandas_dataframe):
  data_df = pandas_dataframe
  data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
  data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
  data_x, data_y = convert_data(data_df)
  return data_x, data_y


SEQ_LEN= 128
BATCH_SIZE = 20
DATA_COLUMN = 'document'
LABEL_COLUMN= 'label'

train_x, train_y= load_data(train)


100%|██████████| 150000/150000 [01:00<00:00, 2465.49it/s]


In [51]:
test_x, test_y = load_data(test)

100%|██████████| 50000/50000 [00:19<00:00, 2500.09it/s]


In [52]:
test_y

array([1, 0, 0, ..., 0, 0, 0])

#### 버트를 활용한 감성분석 모델 만들기

In [53]:
# TPU 객체지정
TPU = True
if TPU :
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
else:
    pass

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.81.204.170:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.81.204.170:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.
